In [52]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
from pathlib import Path
import uuid
import torch
import json
import shutil
import collections
from torch.utils.tensorboard import SummaryWriter
from nnunet.training.model_restore import restore_model
from batchgenerators.utilities.file_and_folder_operations import join

sys.path.insert(0, '../')
sys.path.insert(0, '/executables')
from kaapana_federated.kaapana_federated import KaapanaFederatedTrainingBase, requests_retry_session


In [57]:
class nnUNetFederatedTraining(KaapanaFederatedTrainingBase):

    @staticmethod
    def get_network_trainer(folder):
        checkpoint = join(folder, "model_final_checkpoint.model")
        pkl_file = checkpoint + ".pkl"
        return restore_model(pkl_file, checkpoint, False)

    def __init__(self, run_id=None, workflow_dir=None, federated_operators=None, skip_operators=None):
        super().__init__(workflow_dir)
        
        if self.remote_conf_data['workflow_form']['train_max_epochs'] % self.remote_conf_data['federated_form']['federated_total_rounds'] != 0:
            raise ValueError('train_max_epochs has to be multiple of federated_total_rounds')
        else:
            self.remote_conf_data['workflow_form']['epochs_per_round'] = int(self.remote_conf_data['workflow_form']['train_max_epochs'] / self.remote_conf_data['federated_form']['federated_total_rounds'])

        print(f"Epochs per round {self.remote_conf_data['workflow_form']['epochs_per_round']}")
        
    def tensorboard_logs(self, federated_round):
        current_federated_round_dir = Path(os.path.join(self.fl_working_dir, str(federated_round)))
        for instance_name, _ in self.tmp_federated_site_info.items():
            filename = current_federated_round_dir / instance_name / 'nnunet-training' / 'experiment_results.json'
            with open(filename) as json_file:
                exp_data = json.load(json_file)
            tensorboard_log_dir = Path(os.path.join('/minio', 'tensorboard', self.remote_conf_data["federated_form"]["federated_dir"], os.getenv('OPERATOR_OUT_DIR', 'federated-operator'), instance_name))
            if tensorboard_log_dir.is_dir():
                print('Removing previous logs, since we will write all logs again...')
                shutil.rmtree(tensorboard_log_dir)
            self.writer = SummaryWriter(log_dir=tensorboard_log_dir)
            for epoch_data in exp_data:
                for key, value in epoch_data.items():
                    if key != 'epoch' and key != 'fold':
                        self.writer.add_scalar(key, value, epoch_data['epoch'])
                
    def update_data(self, federated_round):     
        print(Path(os.path.join(self.fl_working_dir, str(federated_round))))
        self.tensorboard_logs(federated_round)
        models_path = Path(os.path.join(self.fl_working_dir, str(federated_round)))
        averaged_state_dict = collections.OrderedDict()
        averaged_amp_grad_scaler = dict()
        print('Loading averaged checkpoints')
        for idx, fname in enumerate(models_path.rglob('model_final_checkpoint.model')):
            print(fname)
            checkpoint = torch.load(fname, map_location=torch.device('cpu'))
            if idx==0:
                for key, value in checkpoint['state_dict'].items():
                    averaged_state_dict[key] = value
                if 'amp_grad_scaler' in checkpoint.keys():
                    for key, value in checkpoint['amp_grad_scaler'].items():
                        averaged_amp_grad_scaler[key] = value 
            else:
                for key, value in checkpoint['state_dict'].items():
                    averaged_state_dict[key] =  (averaged_state_dict[key] + checkpoint['state_dict'][key]) / 2.
                if 'amp_grad_scaler' in checkpoint.keys():
                    for key, value in checkpoint['amp_grad_scaler'].items():
                        averaged_amp_grad_scaler[key] = (averaged_amp_grad_scaler[key] + checkpoint['amp_grad_scaler'][key]) / 2.

        print('Saving averaged checkpoints')
        for idx, fname in enumerate(models_path.rglob('model_final_checkpoint.model')):
            print(fname)
            checkpoint['state_dict'] = averaged_state_dict
#             if 'amp_grad_scaler' in checkpoint.keys():
#                 checkpoint['amp_grad_scaler'] = averaged_amp_grad_scaler
            torch.save(checkpoint, fname)

        self.remote_conf_data['workflow_form']['train_continue'] = True
        print(federated_round, self.remote_conf_data['federated_form']['federated_total_rounds'])
            
kaapana_ft = nnUNetFederatedTraining(run_id='nnunet-federated-220315232240912342')
kaapana_ft.train()

['dkfz', 'hamburg']
